In [1]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, EmailStr, field_validator
from datetime import date
import re
import json


In [2]:
class Subscriber(BaseModel):
    last_name: str
    first_name: str
    birth_date: date
    phone: str
    email: EmailStr

    @field_validator("last_name", "first_name")
    def validate_name(cls, v, info):
        # Только кириллица, с заглавной буквы
        if not re.match(r"^[А-ЯЁ][а-яё]+$", v):
            raise ValueError(f"{info.field_name} должен содержать только кириллицу и начинаться с заглавной буквы")
        return v

    @field_validator("phone")
    def validate_phone(cls, v):
        # Разрешён формат: +7XXXXXXXXXX или 8XXXXXXXXXX
        if not re.match(r"^(\+7|8)\d{10}$", v):
            raise ValueError("Телефон должен быть в формате +7XXXXXXXXXX или 8XXXXXXXXXX")
        return v


In [3]:
app = FastAPI()

@app.post("/subscriber")
def submit_subscriber(sub: Subscriber):
    try:
        # Сохраняем данные в json-файл, каждый раз перезаписываем
        with open("subscriber.json", "w", encoding="utf-8") as f:
            json.dump(sub.dict(), f, ensure_ascii=False, indent=2, default=str)
        return {"status": "success", "data": sub}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
